# UCI Adult Data Set 

The dataset has details of 32561 people with the following 14 features:

[Census Income Dataset](https://archive.ics.uci.edu/ml/datasets/adult)

![](adult.png)

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report , accuracy_score
from sklearn.model_selection import train_test_split
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

In [84]:
data = pd.read_csv("adult1.csv")
data.head()

age         workclass  fnlwgt  education  education-num  \
0   39         State-gov   77516  Bachelors             13   
1   50  Self-emp-not-inc   83311  Bachelors             13   
2   38           Private  215646    HS-grad              9   
3   53           Private  234721       11th              7   
4   28           Private  338409  Bachelors             13   

       marital-status         occupation   relationship   race     sex  \
0       Never-married       Adm-clerical  Not-in-family  White    Male   
1  Married-civ-spouse    Exec-managerial        Husband  White    Male   
2            Divorced  Handlers-cleaners  Not-in-family  White    Male   
3  Married-civ-spouse  Handlers-cleaners        Husband  Black    Male   
4  Married-civ-spouse     Prof-specialty           Wife  Black  Female   

   capital-gain  capital-loss  hours-per-week native-country salary  
0          2174             0              40  United-States  <=50K  
1             0             0              13  United-States  <=50K  
2             0             0              40  United-States  <=50K  
3             0             0              40  United-States  <=50K  
4             0             0              40           Cuba  <=50K

### 1.  Explore the data by completing the following tasks?
- How many men and women (sex feature) are represented in this dataset? 

- What is the average age (age feature) of women? 

- What is the percentage of German citizens (native-country feature)?

In [4]:
data['sex'].value_counts()

Male      21790
Female    10771
Name: sex, dtype: int64

In [20]:
df = data.groupby('sex')['age'].describe()
df['count'].to_dict()

{'Female': 10771.0, 'Male': 21790.0}

In [21]:
df.loc['Female','mean']

36.85823043357163

In [22]:
df.iloc[0,1]

36.85823043357163

In [27]:
data['native-country'].value_counts()['Germany']/data.shape[0]

0.004207487485028101

### 2. For people who recieve more than 50K per year (salary feature), what is the mean and standard deviation of their age?  Simiarly for people who receive less than 50K per year?

In [39]:
dict_1 = (data[data['salary']=='>50K'].describe()['age']).to_dict()
dict_2 = (data[data['salary']=='<=50K'].describe()['age']).to_dict()

In [42]:
dict_1

{'count': 7841.0,
 'mean': 44.24984058155847,
 'std': 10.51902771985177,
 'min': 19.0,
 '25%': 36.0,
 '50%': 44.0,
 '75%': 51.0,
 'max': 90.0}

In [44]:
dict_2

{'count': 24720.0,
 'mean': 36.78373786407767,
 'std': 14.020088490824813,
 'min': 17.0,
 '25%': 25.0,
 '50%': 34.0,
 '75%': 46.0,
 'max': 90.0}

In [46]:
data.groupby('salary').age.agg([np.mean,np.std])

mean        std
salary                      
<=50K   36.783738  14.020088
>50K    44.249841  10.519028

### 3. Display the statistics of age for each gender of all the races (race feature). Use groupby() and describe(). Find the maximum age of men of Amer-Indian-Eskimo race.

In [55]:
data.groupby(['sex','race']).age.agg([np.max]).T['Male']['Amer-Indian-Eskimo']

amax    82
Name: Amer-Indian-Eskimo, dtype: int64

### 4. As computers understand only numbers, we will do some preprocessing tasks for encoding the categorical features.

- Encode Salary column such that wherever we have salary more than 50k that is '>50k' we encode it to 1 else 0. 
- One hot encode the categorical features.
- Split features and target variable into X and y respectively.

In [93]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
data.salary=lb.fit_transform(data['salary'])
#data.salary.map({'<=50K':0,'>50K' : 1})

In [94]:
data.salary.unique()

array([0, 1], dtype=int64)

In [95]:
lb.classes_

array(['<=50K', '>50K'], dtype=object)

In [91]:
#data.salary = lb.inverse_transform(data.salary)

In [57]:
data.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
salary            object
dtype: object

In [100]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
cat_df_ohe = ohe.fit_transform(data.select_dtypes(include='object')).toarray()

In [103]:
cat_df_ohe = pd.DataFrame(cat_df_ohe)
merge_df = pd.concat([data,cat_df_ohe],axis =1)

In [105]:
cat_df = data.select_dtypes(include='object')
merge_df.drop(cat_df.columns,axis =1,inplace =True)

In [107]:
data.head()

age  workclass  fnlwgt  education  education-num      marital-status  \
0   39          7   77516  Bachelors             13       Never-married   
1   50          6   83311  Bachelors             13  Married-civ-spouse   
2   38          4  215646    HS-grad              9            Divorced   
3   53          4  234721       11th              7  Married-civ-spouse   
4   28          4  338409  Bachelors             13  Married-civ-spouse   

          occupation   relationship   race     sex  capital-gain  \
0       Adm-clerical  Not-in-family  White    Male          2174   
1    Exec-managerial        Husband  White    Male             0   
2  Handlers-cleaners  Not-in-family  White    Male             0   
3  Handlers-cleaners        Husband  Black    Male             0   
4     Prof-specialty           Wife  Black  Female             0   

   capital-loss  hours-per-week native-country  salary  
0             0              40  United-States       0  
1             0              13  United-States       0  
2             0              40  United-States       0  
3             0              40  United-States       0  
4             0              40           Cuba       0

In [118]:
lbs_ka_list = dict()

In [119]:
from sklearn.preprocessing import LabelBinarizer
for col in data.select_dtypes(include ='object'):
    lbb = LabelBinarizer()
    lbb.fit(data[col])
    lbs_ka_list.setdefault(col,lbb)

In [120]:
lbs_ka_list

{'education': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'marital-status': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'occupation': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'relationship': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'race': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'sex': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'native-country': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)}

### 5. Perform the following operation on dataset.
- Split the data X and y into X_train,X_test,y_train and y_test in the ratio 70:30
- Further split the training data into train and validation in 80:20 ratio
- Then apply the base Decision Tree Classifier model and calculate the accuracy on validation data as well as on test data.

In [106]:
from sklearn.model_selection import train_test_split as tts

X , y = merge_df.iloc[:,:-1], merge_df.iloc[:,-1]

x_train, x_test, y_train, y_test = tts(X,y,test_size=0.3,random_state =0)

### 6. Perform ensembling using the models Decision Tree Classifier and Logistic Regression, using a VotingClassifier keeping the parameter voting as `soft` and the calculate the accuracy.

## What is the Effect of adding more trees

- Train 3 different ensemble classifiers in the form of gradient boosted trees. Train models with 10, 50 and  100 trees.Use the n_estimators parameter in the boosted tree module.

- n-estimators:
    - The number of sequential trees to be modeled 
    - Though GBM is fairly robust at higher number of trees but it can still overfit at a point.

- Let's get sarted with a model with n_estimators = 10 and max_depth=6:


```python
model_10 = GradientBoostingClassifier(n_estimators=10, max_depth=6).fit(X_train, y_train)
```

### 7. Perform the following Boosting task on below two models.

- Train two models with 
```python
A) n_estimators = 50 & max_depth = 6
B) n_estimators =100 & max_depth= 6
```
- Calculate the accuracy on validation data and testing data.

**Things to ponder**

- Which model has the best accuracy on the validation data?
- Is it always true that the model with the most trees will perform best on validation data?

### 8. Based on the best gradient boosting classifier you found in the previous task, plot a bar plot of the model's top 10 features with it's feature importance score

### 9. Plot the training and testing error vs. number of trees

- Steps to follow:

**Step 1: Calculate the classification error for model on the training data (train_data).**

**Step 2: Store the training errors into a list (called training_errors) that looks like this:**

```python 
[train_err_10, train_err_50, train_err_100]
```
**Step 3: Calculate the classification error of each model on the validation data (validation_data).**

**Step 4: Store the validation classification error into a list (called validation_errors) that looks like this:**

```python
[validation_err_10, validation_err_50,validation_err_100]
```
**Step 5: Calculate the classification error of each model on the test data (test_data).**

**Step 6: Store the testing classification error into a list (called testing_errors) that looks like this:**
```python
[testing_err_10, testing_err_50,testing_err_100]
```

**Things to ponder**

- Does the training error reduce as the number of trees increases?Is it always true that the validation error will reduce as the number of trees increases?